In [191]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings("ignore")


**Loading the dataset**

In [193]:
##Psycopg2 connector
# SQLAlchemy connection string format:
# postgresql+psycopg2://<username>:

DATABASE_URL = (
    "postgresql+psycopg2://neondb_owner:npg_CeS9fJg2azZD"
    "@ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech:5432/neondb"
    "?sslmode=require"
)
# Create engine and establish connection
engine = create_engine(DATABASE_URL)

In [194]:
import pandas as pd
df=pd.read_csv('sat-results.csv')
df

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,SAT Critical Readng Avg. Score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363,355,218160,x345,78%,2.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366,383,268547,x234,NaN,3.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370,377,236446,x123,NaN,3.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359,414,427826,x123,92%,4.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384,390,672714,x123,92%,2.0
...,...,...,...,...,...,...,...,...,...,...,...
488,27Q480,JOHN ADAMS HIGH SCHOOL,403,391,409,392,391,863765,NaN,92%,1.0
489,13K605,GEORGE WESTINGHOUSE CAREER AND TECHNICAL EDUCA...,85,406,391,392,406,937579,x234,NaN,NaN
490,05M304,MOTT HALL HIGH SCHOOL,54,413,399,398,413,296405,x123,78%,2.0
491,02M520,MURRY BERGTRAUM HIGH SCHOOL FOR BUSINESS CAREERS,264,407,440,393,407,892839,NaN,92%,2.0


# Cleaning Headers

**Normalising Headers**

In [195]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('.', '')

print("\nHeaders after normalising:")
print(df.columns)


Headers after normalising:
Index(['dbn', 'school_name', 'num_of_sat_test_takers',
       'sat_critical_reading_avg_score', 'sat_math_avg_score',
       'sat_writing_avg_score', 'sat_critical_readng_avg_score',
       'internal_school_id', 'contact_extension', 'pct_students_tested',
       'academic_tier_rating'],
      dtype='object')


**Typo Error**

In [196]:
df = df.rename(columns={'sat_critical_readng_avg_score': 'sat_critical_reading_avg_score_new'})

# Cleaning data type and values

In [197]:
# Convert 'num_of_sat_test_takers' to float
df['num_of_sat_test_takers'] = pd.to_numeric(df['num_of_sat_test_takers'], errors='coerce')

# Convert 'pct_students_tested' to float
df['pct_students_tested'] = df['pct_students_tested'].str.replace('%', '').replace('N/A', np.nan)
df['pct_students_tested'] = pd.to_numeric(df['pct_students_tested'], errors='coerce')

# Convert 'academic_tier_rating' to float
df['academic_tier_rating'] = pd.to_numeric(df['academic_tier_rating'], errors='coerce')

# Convert SAT scores to float and validate range (200-800)
sat_score_cols = ['sat_critical_reading_avg_score', 'sat_critical_reading_avg_score_new','sat_math_avg_score', 'sat_writing_avg_score']
for col in sat_score_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    invalid_scores = (df[col] < 200) | (df[col] > 800)
    df.loc[invalid_scores, col] = np.nan

# Dropping Duplicates and Unrelated Fields

**Drop Duplicate Columns**

In [198]:
# Select both columns by their unique names
col_correct = df['sat_critical_reading_avg_score']
col_new = df['sat_critical_reading_avg_score_new']

# Count the number of nulls in each column
nulls_correct = col_correct.isnull().sum()
nulls_new = col_new.isnull().sum()

# Determine which column to drop based on the null count
if nulls_new > nulls_correct:
    # Drop the new column if it has more nulls
    df.drop(columns=['sat_critical_reading_avg_score_new'], inplace=True)
    print("Dropped 'sat_critical_reading_avg_score_new' due to more null values.")
else:
    # Drop the original column if it has more nulls or the same number of nulls
    df.drop(columns=['sat_critical_reading_avg_score'], inplace=True)
    print("Dropped 'sat_critical_reading_avg_score' due to more nulls or equal nulls.")
    # The 'new' column now contains the most complete data, so we rename it
    df.rename(columns={'sat_critical_reading_avg_score_new': 'sat_critical_reading_avg_score'}, inplace=True)
    print("Renamed the remaining column to 'sat_critical_reading_avg_score'.")


Dropped 'sat_critical_reading_avg_score' due to more nulls or equal nulls.
Renamed the remaining column to 'sat_critical_reading_avg_score'.


**Drop Duplicate Value**

In [199]:
new_df=df.drop_duplicates()
new_df

,dbn,school_name,num_of_sat_test_takers,sat_math_avg_score,sat_writing_avg_score,sat_critical_reading_avg_score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29.0,404.0,363.0,355.0,218160,x345,78.0,2.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91.0,423.0,366.0,383.0,268547,x234,NaN,3.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70.0,402.0,370.0,377.0,236446,x123,NaN,3.0
3,01M458,FORSYTH SATELLITE ACADEMY,7.0,401.0,359.0,414.0,427826,x123,92.0,4.0
4,01M509,MARTA VALLE HIGH SCHOOL,44.0,433.0,384.0,390.0,672714,x123,92.0,2.0
...,...,...,...,...,...,...,...,...,...,...
473,75X012,P.S. X012 LEWIS AND CLARK SCHOOL,NaN,NaN,NaN,NaN,733698,x234,92.0,1.0
474,75X754,J. M. RAPPORT SCHOOL CAREER DEVELOPMENT,NaN,NaN,NaN,NaN,976034,x345,NaN,4.0
475,79M645,SCHOOL FOR COOPERATIVE TECHNICAL EDUCATION,NaN,NaN,NaN,NaN,556924,x123,85.0,1.0
476,79Q950,GED PLUS s CITYWIDE,8.0,400.0,426.0,496.0,544514,x234,92.0,2.0


**Drop Unrelated Fields**

To keep the data focussed, removing columns 'internal_school_id' and 'contact_extension' as it is mentioned that they are potentially generated/unchecked fields.

In [201]:
cols_to_drop = ['internal_school_id', 'contact_extension']

for col in cols_to_drop:
    if col in df.columns:
        df.drop(columns=[col], inplace=True)
        print(f"Dropped '{col}' column.")
    else:
        print(f"Column '{col}' not found in DataFrame.")

print("\nRemaining columns:")
print(df.columns)

Column 'internal_school_id' not found in DataFrame.
Column 'contact_extension' not found in DataFrame.

Remaining columns:
Index(['dbn', 'school_name', 'num_of_sat_test_takers', 'sat_math_avg_score',
       'sat_writing_avg_score', 'sat_critical_reading_avg_score',
       'pct_students_tested', 'academic_tier_rating'],
      dtype='object')


# Cleaning the data

In [202]:
print(df.head())

      dbn                                    school_name  \
0  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES   
1  01M448            UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   
2  01M450                     EAST SIDE COMMUNITY SCHOOL   
3  01M458                      FORSYTH SATELLITE ACADEMY   
4  01M509                        MARTA VALLE HIGH SCHOOL   

   num_of_sat_test_takers  sat_math_avg_score  sat_writing_avg_score  \
0                    29.0               404.0                  363.0   
1                    91.0               423.0                  366.0   
2                    70.0               402.0                  370.0   
3                     7.0               401.0                  359.0   
4                    44.0               433.0                  384.0   

   sat_critical_reading_avg_score  pct_students_tested  academic_tier_rating  
0                           355.0                 78.0                   2.0  
1                           383.0               

In [203]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dbn                             493 non-null    object 
 1   school_name                     493 non-null    object 
 2   num_of_sat_test_takers          435 non-null    float64
 3   sat_math_avg_score              430 non-null    float64
 4   sat_writing_avg_score           435 non-null    float64
 5   sat_critical_reading_avg_score  435 non-null    float64
 6   pct_students_tested             376 non-null    float64
 7   academic_tier_rating            402 non-null    float64
dtypes: float64(6), object(2)
memory usage: 30.9+ KB
None


**Validating SAT Scores:** The SAT scores should be within the 200-800 range. Converting them to numbers and handling invalid scores.

In [204]:
sat_score_cols = ['sat_critical_reading_avg_score', 'sat_math_avg_score', 'sat_writing_avg_score']

for col in sat_score_cols:
    # Convert column to a numeric type, setting non-numeric values to NaN
    df[col] = pd.to_numeric(df[col], errors='coerce')

    # Identify and replace scores outside the valid range (200-800) with NaN
    invalid_scores_mask = (df[col] < 200) | (df[col] > 800)
    df.loc[invalid_scores_mask, col] = np.nan
    print(f"Cleaned and validated {col}.")

Cleaned and validated sat_critical_reading_avg_score.
Cleaned and validated sat_math_avg_score.
Cleaned and validated sat_writing_avg_score.


**Handling Missing Values (NaN)**

In [205]:
df.dropna(subset=sat_score_cols, how='all', inplace=True)
print("\nDropped rows where all SAT scores were missing.")


Dropped rows where all SAT scores were missing.


In [206]:
# Final state of the DataFrame
print("\nFinal Cleaned DataFrame Info:")
df.info()
print("\nFinal 5 rows:")
print(df.head())


Final Cleaned DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 435 entries, 0 to 492
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dbn                             435 non-null    object 
 1   school_name                     435 non-null    object 
 2   num_of_sat_test_takers          435 non-null    float64
 3   sat_math_avg_score              430 non-null    float64
 4   sat_writing_avg_score           435 non-null    float64
 5   sat_critical_reading_avg_score  435 non-null    float64
 6   pct_students_tested             329 non-null    float64
 7   academic_tier_rating            361 non-null    float64
dtypes: float64(6), object(2)
memory usage: 30.6+ KB

Final 5 rows:
      dbn                                    school_name  \
0  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES   
1  01M448            UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   
2  01M450   

In [207]:
df

,dbn,school_name,num_of_sat_test_takers,sat_math_avg_score,sat_writing_avg_score,sat_critical_reading_avg_score,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29.0,404.0,363.0,355.0,78.0,2.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91.0,423.0,366.0,383.0,NaN,3.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70.0,402.0,370.0,377.0,NaN,3.0
3,01M458,FORSYTH SATELLITE ACADEMY,7.0,401.0,359.0,414.0,92.0,4.0
4,01M509,MARTA VALLE HIGH SCHOOL,44.0,433.0,384.0,390.0,92.0,2.0
...,...,...,...,...,...,...,...,...
488,27Q480,JOHN ADAMS HIGH SCHOOL,403.0,409.0,392.0,391.0,92.0,1.0
489,13K605,GEORGE WESTINGHOUSE CAREER AND TECHNICAL EDUCA...,85.0,391.0,392.0,406.0,NaN,NaN
490,05M304,MOTT HALL HIGH SCHOOL,54.0,399.0,398.0,413.0,78.0,2.0
491,02M520,MURRY BERGTRAUM HIGH SCHOOL FOR BUSINESS CAREERS,264.0,440.0,393.0,407.0,92.0,2.0


In [208]:
# Cleaning the DataFrame to a new CSV file
df.to_csv('cleaned_nyc_sat_results.csv', index=False)
print("\nCleaned data successfully saved to 'cleaned_nyc_sat_results.csv'.")


Cleaned data successfully saved to 'cleaned_nyc_sat_results.csv'.


**Desinging the schema**

In [209]:
new_df.to_sql(
    name='deepshikha_sat_results',       
    con=engine,     
    schema='nyc_schools',
    if_exists='replace',    
    index=False            
)

478